In [1]:
import pandas as pd
import numpy as np
import scipy
import nltk
import spacy
import gensim
import glob
import csv
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import cross_val_score
import sklearn.model_selection
import sklearn.pipeline
import re
from sklearn import svm
from sklearn import *
from sklearn.feature_selection import SelectKBest, VarianceThreshold
from sklearn.feature_selection import chi2
from sklearn.base import BaseEstimator, TransformerMixin
import gensim.models.wrappers.fasttext
from scipy import sparse
import tensorflow_datasets as tfds
import tensorflow as tf
import collections
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import LeaveOneOut,KFold,train_test_split
import ktrain
from ktrain import text
from sklearn.metrics import accuracy_score

# Custom imports
from mr_generic_scripts import *
from mr_cls_Transformer import *

In [2]:
# Path to xlsx files folder
path_to_raw_files = 'Data/raw/'

In [3]:
# Get the datasets in dataframes
a_dataset = mr_get_data(path_to_raw_files)
qa_dataset = mr_get_qa_data(path_to_raw_files)

In [4]:
# Sanity check
print(a_dataset[0][1][:5])
print(qa_dataset[0][1][:5])
print(len(a_dataset[0][1]))

   Child_ID                                             Answer  Score  Age  \
0         0   Because they did n't want the woman to find them      2    9   
1         1           Because they Were n't supose to be there      0    9   
2         2                                they have a seccret      0    9   
3         3               because they didnt whant to get cort      1    9   
4         4  Because they did n't want the woman to see them .      1    9   

   Gender           Question  
0       0  SFQuestion_1_Text  
1       1  SFQuestion_1_Text  
2       0  SFQuestion_1_Text  
3       0  SFQuestion_1_Text  
4       1  SFQuestion_1_Text  
   Child_ID                                             Answer  Score  Age  \
0         0  Why did the men hide ? Because they did n't wa...      2    9   
1         1  Why did the men hide ? Because they Were n't s...      0    9   
2         2         Why did the men hide ? they have a seccret      0    9   
3         3  Why did the men hide

In [5]:
tr_cls = MR_transformer(text_cols,[7,8,9,10,11,12,13,14],[0,1,2],20)
tr_cls.mr_set_eval_vars(True,True,False)

In [6]:
def mr_proc_results(raw_results):
    # Process the results from the 10 runs
    # result format: [acc, acc per q, acc per age], [f1, f1 per q, f1 per age]
    # Ignore ages as they seem to be mostly consistent with global average
    # Ignore accs per question and age as averaging them seems to be consistent with global average
    # Report global acc, global macro f1, average of macro f1 per question
    pr_results = [[acc_score, f1_score,round(sum(qa_s)/11,2),round(sum(qf_s)/11,2)] 
                for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    
    # Throw the list in an np array
    pr_arr = np.array(pr_results)

    # Print the results
    pp = pprint.PrettyPrinter(indent=4)

    print("Acc, F1, Avg-F1-Per-Q")
    pp.pprint(pr_results)
    pp.pprint(np.mean(pr_arr,axis=0))
    
    
    print("\nAcc per question (avg over 10 runs)")
    # Get the result per question
    pr_qa_results = [[qa_s] for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    pr_qa_arr = np.array(pr_qa_results)
    avg_qa = np.mean(pr_qa_arr,axis=0).tolist()
    print(' & '.join([str(round(res,2)) for res in avg_qa[0]]))
    #pp.pprint(np.mean(pr_qa_arr,axis=0))
    
    print("\nF1 per question (avg over 10 runs)")
    pr_qf_results = [[qf_s] for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s]) in raw_results]
    pr_qf_arr = np.array(pr_qf_results)
    avg_qf = np.mean(pr_qf_arr,axis=0).tolist()
    print(' & '.join([str(round(res,2)) for res in avg_qf[0]]))
    #pp.pprint(np.mean(pr_qf_arr,axis=0))

In [7]:
# 10 fold cross validation, original dataset, without the question
# 25% split for validation (each run is 67.5 / 22.5 / 10 for train / val / test)
results_1 = tr_cls.mr_kfold_train_test(a_dataset[-1][1],0.25,10)

preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 20
	99percentile : 27


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 20
	99percentile : 26




begin training using onecycle policy with max lr of 8e-05...
Train for 1273 steps, validate for 80 steps
Epoch 1/4
1273/1273 [==============================] - 83s 65ms/step - loss: 0.6365 - accuracy: 0.7384 - val_loss: 0.4630 - val_accuracy: 0.8248
Epoch 2/4
1273/1273 [==============================] - 73s 57ms/step - loss: 0.3629 - accuracy: 0.8714 - val_loss: 0.5685 - val_accuracy: 0.7980
Epoch 3/4
1273/1273 [==============================] - 73s 57ms/step - loss: 0.2510 - accuracy: 0.9121 - val_loss: 0.3627 - val_accuracy: 0.8853
Epoch 4/4
1273/1273 [==============================] - 73s 57ms/step - loss: 0.0971 - accuracy: 0.9671 - val_loss: 0.3882 - val_accuracy: 0.8927
              precision    recall  f1-score   support

           0       0.87      0.86      0.87       768
           1       0.89      0.87      0.88       863
           2       0.92      0.94      0.93       914

    accuracy                           0.89      2545
   macro avg       0.89      0.89      0.

Test Accuracy: 0.89 

Test Macro F1: 0.89 

Evaluating column Question with value SFQuestion_1_Text


Accuracy: 0.91 

Macro F1: 0.85 

Evaluating column Question with value SFQuestion_2_Text


...

Accuracy: 0.89 

Macro F1: 0.88 

Evaluating column Age with value 11


Accuracy: 0.85 

Macro F1: 0.85 

Evaluating column Age with value 12


Accuracy: 0.91 

Macro F1: 0.91 

Evaluating column Age with value 13


Accuracy: 0.89 

Macro F1: 0.86 

Evaluating column Age with value 14


Accuracy: 1.0 

Macro F1: 1.0 



In [9]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results_1)

[   (   [   0.89,
            [0.91, 0.86, 0.94, 0.83, 0.84, 0.82, 0.92, 0.91, 0.87, 0.93, 0.96],
            [0.89, 0.81, 0.89, 0.9, 0.89, 0.87, 0.94, 1.0]],
        [   0.89,
            [0.85, 0.8, 0.9, 0.82, 0.83, 0.79, 0.85, 0.72, 0.84, 0.87, 0.94],
            [0.62, 0.81, 0.89, 0.9, 0.89, 0.86, 0.93, 1.0]]),
    (   [   0.89,
            [0.85, 0.87, 0.9, 0.87, 0.86, 0.86, 0.94, 0.95, 0.84, 0.93, 0.88],
            [0.78, 0.84, 0.89, 0.89, 0.9, 0.88, 0.84, 1.0]],
        [   0.88,
            [0.78, 0.82, 0.84, 0.87, 0.82, 0.85, 0.84, 0.64, 0.83, 0.89, 0.87],
            [0.74, 0.85, 0.9, 0.89, 0.89, 0.88, 0.83, 1.0]]),
    (   [   0.88,
            [0.95, 0.84, 0.87, 0.86, 0.85, 0.85, 0.93, 0.91, 0.84, 0.94, 0.85],
            [0.83, 0.85, 0.89, 0.9, 0.86, 0.85, 0.9, 1.0]],
        [   0.88,
            [0.94, 0.79, 0.78, 0.86, 0.81, 0.79, 0.82, 0.82, 0.81, 0.9, 0.85],
            [0.79, 0.85, 0.89, 0.9, 0.85, 0.84, 0.89, 1.0]]),
    (   [   0.88,
            [0.86, 0.86, 0.85,

In [10]:
mr_proc_results(results_1)

Acc, F1, Avg-F1-Per-Q
[   [0.89, 0.89, 0.89, 0.84],
    [0.89, 0.88, 0.89, 0.82],
    [0.88, 0.88, 0.88, 0.83],
    [0.88, 0.88, 0.88, 0.82],
    [0.89, 0.89, 0.89, 0.81],
    [0.89, 0.89, 0.89, 0.84],
    [0.88, 0.88, 0.88, 0.83],
    [0.89, 0.89, 0.89, 0.84],
    [0.89, 0.89, 0.89, 0.84],
    [0.89, 0.88, 0.88, 0.83]]
array([0.887, 0.885, 0.886, 0.83 ])

Acc per question (avg over 10 runs)
0.88 & 0.86 & 0.91 & 0.86 & 0.88 & 0.84 & 0.92 & 0.92 & 0.85 & 0.92 & 0.89

F1 per question (avg over 10 runs)
0.83 & 0.8 & 0.86 & 0.84 & 0.85 & 0.8 & 0.84 & 0.75 & 0.82 & 0.87 & 0.87


In [11]:
# change the model parameters to set max sent len to 35
tr_cls.mr_set_model_vars(text_cols,[7,8,9,10,11,12,13,14],[0,1,2],35)

In [12]:
results_2 = tr_cls.mr_kfold_train_test(qa_dataset[-1][1],0.25,10)

preprocessing train...
language: en
train sequence lengths:
	mean : 17
	95percentile : 28
	99percentile : 34


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 18
	95percentile : 28
	99percentile : 34




begin training using onecycle policy with max lr of 8e-05...
Train for 1273 steps, validate for 80 steps
Epoch 1/4
1273/1273 [==============================] - 95s 75ms/step - loss: 0.5766 - accuracy: 0.7717 - val_loss: 0.3499 - val_accuracy: 0.8715
Epoch 2/4
1273/1273 [==============================] - 84s 66ms/step - loss: 0.3273 - accuracy: 0.8864 - val_loss: 0.3607 - val_accuracy: 0.8680
Epoch 3/4
1273/1273 [==============================] - 85s 67ms/step - loss: 0.2232 - accuracy: 0.9280 - val_loss: 0.2844 - val_accuracy: 0.9143
Epoch 4/4
1273/1273 [==============================] - 85s 67ms/step - loss: 0.0942 - accuracy: 0.9701 - val_loss: 0.2850 - val_accuracy: 0.9218
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       748
           1       0.89      0.93      0.91       874
           2       0.95      0.95      0.95       923

    accuracy                           0.92      2545
   macro avg       0.92      0.92      0.

Test Accuracy: 0.91 

Test Macro F1: 0.91 

Evaluating column Question with value SFQuestion_1_Text


Accuracy: 0.93 

Macro F1: 0.9 

Evaluating column Question with value SFQuestion_2_Text


...

Accuracy: 0.9 

Macro F1: 0.89 

Evaluating column Age with value 11


Accuracy: 0.9 

Macro F1: 0.89 

Evaluating column Age with value 12


Accuracy: 0.92 

Macro F1: 0.92 

Evaluating column Age with value 13


Accuracy: 0.92 

Macro F1: 0.9 

Evaluating column Age with value 14


Accuracy: 1.0 

Macro F1: 1.0 



In [13]:
pp.pprint(results_2)

[   (   [   0.91,
            [0.93, 0.89, 0.95, 0.84, 0.89, 0.85, 0.92, 0.93, 0.91, 0.92, 0.95],
            [1.0, 0.84, 0.89, 0.91, 0.93, 0.94, 0.93, 1.0]],
        [   0.91,
            [0.9, 0.82, 0.9, 0.79, 0.89, 0.8, 0.79, 0.79, 0.89, 0.85, 0.94],
            [1.0, 0.84, 0.89, 0.91, 0.93, 0.94, 0.92, 1.0]]),
    (   [   0.92,
            [0.92, 0.87, 0.95, 0.92, 0.94, 0.85, 0.94, 0.94, 0.94, 0.93, 0.9],
            [0.89, 0.86, 0.91, 0.92, 0.94, 0.9, 0.94, 1.0]],
        [   0.92,
            [0.92, 0.82, 0.94, 0.9, 0.93, 0.83, 0.84, 0.64, 0.94, 0.89, 0.89],
            [0.82, 0.87, 0.92, 0.92, 0.93, 0.89, 0.94, 1.0]]),
    (   [   0.91,
            [0.94, 0.87, 0.94, 0.9, 0.88, 0.92, 0.93, 0.94, 0.88, 0.94, 0.91],
            [1.0, 0.88, 0.92, 0.93, 0.9, 0.89, 0.92, 1.0]],
        [   0.91,
            [0.91, 0.81, 0.87, 0.88, 0.86, 0.89, 0.83, 0.9, 0.86, 0.89, 0.9],
            [1.0, 0.88, 0.92, 0.92, 0.89, 0.88, 0.91, 1.0]]),
    (   [   0.91,
            [0.94, 0.88, 0.91, 0.

In [14]:
mr_proc_results(results_2)

Acc, F1, Avg-F1-Per-Q
[   [0.91, 0.91, 0.91, 0.85],
    [0.92, 0.92, 0.92, 0.87],
    [0.91, 0.91, 0.91, 0.87],
    [0.91, 0.91, 0.91, 0.87],
    [0.91, 0.91, 0.91, 0.84],
    [0.92, 0.92, 0.92, 0.86],
    [0.92, 0.92, 0.92, 0.88],
    [0.91, 0.91, 0.91, 0.88],
    [0.93, 0.93, 0.93, 0.9],
    [0.91, 0.91, 0.91, 0.87]]
array([0.915, 0.915, 0.915, 0.869])

Acc per question (avg over 10 runs)
0.93 & 0.88 & 0.94 & 0.89 & 0.91 & 0.89 & 0.94 & 0.94 & 0.9 & 0.93 & 0.93

F1 per question (avg over 10 runs)
0.9 & 0.82 & 0.9 & 0.86 & 0.89 & 0.87 & 0.84 & 0.8 & 0.88 & 0.88 & 0.91
